In [25]:
%reset -f

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import time
import tools

In [26]:
# Parameters to tweak
lr = 1e-3
batchSize = 64
epochs = 30
loss_fn = nn.CrossEntropyLoss()

In [27]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# Create 64 batches
train_dataloader = DataLoader(training_data, batch_size=batchSize)
test_dataloader = DataLoader(test_data, batch_size=batchSize)
len(test_dataloader)

157

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [30]:
def train_data(model, optimizer):
    for xb, yb in train_dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    #print(f"Train loss: {loss:>7f}")

In [31]:
def test_data(model):
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss, corrects = 0, 0

    with torch.no_grad():
        for xb, yb in test_dataloader:
            preds = model(xb)
            test_loss += loss_fn(preds, yb).item()
            corrects += (preds.argmax(1) == yb).type(torch.float).sum().item()

    test_loss /= num_batches
    # test_loss = lo
    corrects /= size
    print(f"Accuracy: {(100*corrects):>0.1f}%, Avg loss: {test_loss:>8f}")
    return corrects

In [32]:
epoch_pro = []
epoch_performance = []
for n in range(0, 33):
    print(f"Started run {n} / {33}")

    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for t in range(epochs):
        start_time = time.time()

        train_data(model, optimizer)
        epoch_pro.append(test_data(model))
        epoch_performance.append(time.time()-start_time)


Started run 0 / 33
Accuracy: 28.5%, Avg loss: 2.265891
Accuracy: 43.5%, Avg loss: 2.229743
Accuracy: 51.4%, Avg loss: 2.191668
Accuracy: 55.9%, Avg loss: 2.150496
Accuracy: 58.9%, Avg loss: 2.105220
Accuracy: 61.3%, Avg loss: 2.055059
Accuracy: 63.5%, Avg loss: 1.999535
Accuracy: 65.2%, Avg loss: 1.938564
Accuracy: 66.5%, Avg loss: 1.872513
Accuracy: 67.9%, Avg loss: 1.802196
Accuracy: 69.2%, Avg loss: 1.728799
Accuracy: 70.5%, Avg loss: 1.653728
Accuracy: 71.7%, Avg loss: 1.578451
Accuracy: 73.0%, Avg loss: 1.504338
Accuracy: 74.0%, Avg loss: 1.432551
Accuracy: 75.0%, Avg loss: 1.363981
Accuracy: 76.0%, Avg loss: 1.299233
Accuracy: 77.0%, Avg loss: 1.238648
Accuracy: 77.7%, Avg loss: 1.182343
Accuracy: 78.5%, Avg loss: 1.130271
Accuracy: 79.2%, Avg loss: 1.082271
Accuracy: 79.9%, Avg loss: 1.038109
Accuracy: 80.4%, Avg loss: 0.997515
Accuracy: 81.0%, Avg loss: 0.960203
Accuracy: 81.6%, Avg loss: 0.925892
Accuracy: 82.0%, Avg loss: 0.894309
Accuracy: 82.5%, Avg loss: 0.865203
Accuracy:

In [33]:
tools.saveResult(epochs,lr,"crossentropyloss",batchSize,epoch_pro,epoch_performance, "model_cross_entropy_sigmoid_n")